# Test: the impact of an AGN power-law continuum
---
What effect does the inclusion of an AGN power-law continuum have upon the recovered SFH?

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [4]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd
import extinction

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_cumulative_mass, compute_cumulative_light
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted, ppxf_plot
from ppxftests.plot_ppxf_summary import plot_ppxf_summary

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


## Test: can we fit an AGN continuum in ppxf using the `sky` keyword?
---
Need to test:
* How well does ppxf recover the slope/amplitude of the AGN continuum? Test on a combination of amplitudes/slopes.
* What happens if we apply extinction to the mock spectrum? Does ppxf do a good job of returning a multiplicative polynomial with the right shape?

What plots do we want?
For single runs: 
* ppxf output plot 
* comparison of input/output SFHs (lift this code from somewhere else)
* Histograms showing distribution of "sky" template weights; vertical line showing input weight


In [28]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  fit_agn_cont=False,
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [29]:
###############################################################################
# Load a realistic SFH
###############################################################################
plt.close("all")
isochrones = "Padova"
SNR = 100
z = 0
niters = 20
nthreads = 20

gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw_input)

x_AGN = 1.0
alpha_nu = 1.5
lambda_norm_A = 4020

###############################################################################
# Run ppxf without an AGN continuum added as a "control"
###############################################################################
# Create spectrum
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=True, x_AGN=x_AGN, alpha_nu=alpha_nu,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

# Get the AGN continuum (for plotting purposes)
alpha_lambda = 2 - alpha_nu
lambda_norm_idx = np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))
F_star_0 = spec[lambda_norm_idx] / (1 + x_AGN)  # Note that this may be slightly wrong because noise has been added to spec
F_agn_0 = x_AGN * F_star_0
F_lambda_0 = F_agn_0 / lambda_norm_A**(-alpha_lambda)
spec_agn = F_lambda_0 * lambda_vals_A**(-alpha_lambda)

###########################################################################
# Run ppxf using a MC approach
###########################################################################
# Input arguments
seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Total time in ppxf: {time() - t:.2f} s")

###########################################################################
# Run ppxf with regularisation
###########################################################################
t = time()
pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="auto",
              fit_agn_cont=False,
              isochrones=isochrones,
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1, regul_start=4.5e4,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

###########################################################################
# Summary plots
###########################################################################
plot_ppxf_summary(sfh_lw_input, sfh_mw_input, pp_regul, pp_mc_list, isochrones)

# Check the quality of the fit
ppxf_plot(pp_regul)
plt.plot(lambda_vals_A, spec_agn / pp_regul.norm, label="AGN continuum (input)")
plt.legend()
ppxf_plot(pp_mc_list[0])
plt.plot(lambda_vals_A, spec_agn / pp_regul.norm, label="AGN continuum (input)")
plt.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## What happens if we apply extinction?
---
Concerns:
* Without including an AGN continuum, how does ppxf fare in accurately recovering the SFH if mpoly is used vs. an extinction correction?

In [6]:
###############################################################################
# Load a realistic SFH
###############################################################################
isochrones = "Padova"
SNR = 100
z = 0
niters = 100
nthreads = 20

gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw_input)

###############################################################################
# Create spectrum
###############################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=True, x_AGN=0.1, alpha_nu=1.5,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

# Create second spectrum without an AGN continuum 
spec_noagn, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

# Obtain the AGN spectrum on its own
spec_agn = spec - spec_noagn

# Apply extinction
A_V = 1.0
A_vals = extinction.fm07(lambda_vals_A, a_v=A_V, unit="aa")
ext_factor = 10**(-0.4 * A_vals)

spec_ext = spec * ext_factor
spec_ext_err = spec_err * ext_factor
spec_ext_agn = spec_agn * ext_factor

# Plot 
fig, axs = plt.subplots(nrows=2, figsize=(15, 7.5))
axs[0].plot(lambda_vals_A, spec, label="Stellar continuum - before applying extinction")
axs[0].plot(lambda_vals_A, spec_ext, label="Stellar continuum - after applying extinction")
axs[0].plot(lambda_vals_A, spec_agn, label="Stellar continuum - before applying extinction")
axs[0].plot(lambda_vals_A, spec_ext_agn, label="Stellar continuum - after applying extinction")
axs[1].plot(lambda_vals_A, ext_factor, label="Extinction factor")
[ax.legend() for ax in axs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach
###########################################################################
# Input arguments
seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Total time in ppxf: {time() - t:.2f} s")

###########################################################################
# Run ppxf with regularisation
###########################################################################
t = time()
pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="auto",
              fit_agn_cont=True,
              isochrones=isochrones,
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

###########################################################################
# Summary plots
###########################################################################
plot_ppxf_summary(sfh_lw_input, sfh_mw_input, pp_regul, pp_mc_list, isochrones)

# Check the quality of the fit
ppxf_plot(pp_regul)
plt.gcf().plot(lambda_vals_A, spec_agn, label="AGN continuum (input)")
plt.gcf().legend()
ppxf_plot(pp_mc_list[0])
plt.gcf().plot(lambda_vals_A, spec_agn, label="AGN continuum (input)")
plt.gcf().legend()

In [82]:
###########################################################################
# Parse the output
###########################################################################
# Print weights of "sky templates"
nskytemps = len(pp_mc_list[0].weights[pp_mc_list[0].ntemp:])
for pp in pp_mc_list:
    print(pp.weights[pp.ntemp:])

# Histograms showing the weights of the sky templates
fig, ax = plt.subplots()
for aa in range(nskytemps):
    ax.hist([pp.weights[pp.ntemp:][aa] for pp in pp_mc_list], range=(0, 1), bins=20, histtype="step", label=aa)
ax.legend()
ax.set_xlabel("AGN template weight")
ax.set_ylabel(r"$N$")
    

[0.         0.         0.33595267 0.00840045]
[0.         0.         0.29255563 0.04007049]
[0.         0.         0.33287152 0.01067401]
[0.         0.         0.34090081 0.00387884]
[0.         0.         0.29818487 0.04054532]
[0.         0.         0.31762147 0.01236431]
[0.        0.        0.3409706 0.       ]
[0.         0.         0.31611929 0.01681322]
[0.         0.         0.32489925 0.02211988]
[0.         0.         0.24939663 0.08382288]
[0.         0.         0.25051072 0.08065105]
[0.         0.         0.33545208 0.00231557]
[0.         0.         0.27169184 0.06453099]
[0.         0.         0.24772024 0.08503345]
[0.         0.         0.33655061 0.0115635 ]
[0.         0.         0.33799889 0.00492616]
[0.         0.         0.28613719 0.05240256]
[0.         0.         0.27195088 0.05965296]
[0.         0.         0.31914658 0.01868204]
[0.         0.         0.25556485 0.07590343]
[0.         0.         0.27193773 0.05835284]
[0.         0.         0.32361848 0.01

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$N$')

# PPXF: Varying both $L_{\rm NT}$ and $\alpha_\nu$
---

In [62]:
###############################################################################
# Settings
###############################################################################
isochrones = "Padova"
SNR = 100
z = 0.01

# Get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)
bin_edges, bin_widths = get_bin_edges_and_widths(isochrones=isochrones)

# ppxf settings
niters = 100
nthreads = 20

# For analysis
lambda_norm_A = 5000
age_thresh_vals = [None, 1e9, None]

# Parameters
# alpha_nu_vals = np.linspace(0.3, 2.1, 5)  # What is a typical value for low-z Seyfert galaxies?
# log_L_NT_vals = np.linspace(42, 44, 5)
alpha_nu_vals = [0.3, 2.0]
log_L_NT_vals = [42, 43, 44]


In [66]:
###############################################################################
# Load a realistic SFH
###############################################################################
gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw_input)

# Create the spectrum here, just so we can check it looks OK
create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.21216727e+40, 1.79949774e+40, 1.93929231e+40, ...,
        2.10821502e+40, 2.11309114e+40, 1.78056878e+40]),
 array([1.20447209e+38, 1.81941317e+38, 1.90595754e+38, ...,
        2.11493125e+38, 2.09762169e+38, 1.79820800e+38]),
 array([3500.        , 3500.77462622, 3501.54925243, ..., 6998.98661775,
        6999.76124396, 7000.53587018]))

In [67]:
###############################################################################
# Before we run ppxf, plot the spectra
###############################################################################
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 4))
ax.set_xlabel("Observed wavelength (Å)")
ax.set_ylabel(r"$F_\lambda(\lambda)\,(erg\,s^{-1}\,Å^{-1})$")

# Plot spectrum without an AGN continuum added as a "control"
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)
lambda_norm_idx = np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))

# Add to plot 
ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, label="No AGN continuum", color="k", zorder=10000, linewidth=0.5)

for alpha_nu, log_L_NT in product(alpha_nu_vals, log_L_NT_vals):
    # Create spectrum
    L_NT = 10**log_L_NT
    spec, spec_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_input,
        agn_continuum=True, L_NT_erg_s=L_NT, alpha_nu=alpha_nu,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)

    # Add to plot 
    ax.errorbar(x=lambda_vals_A, y=spec, yerr=spec_err, 
                label=r"$\alpha_\nu = %.1f, L_{\rm NT} = 10^{%.1f} \,\rm erg\,s^{-1}$" % (alpha_nu, log_L_NT))

ax.legend(fontsize="x-small")
ax.set_yscale("log")
ax.autoscale(axis="x", tight=True, enable=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Run ppxf with an added AGN continuum to investigate its effect on the recovered light/mass-weighted ages
---

In [16]:
###############################################################################
# Run ppxf without an AGN continuum added as a "control"
###############################################################################
# Create spectrum
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach
###########################################################################
# Input arguments
seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Total time in ppxf: {time() - t:.2f} s")

###########################################################################
# Run ppxf with regularisation
###########################################################################
t = time()
pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              regularisation_method="auto",
              isochrones=isochrones,
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")


Running ppxf on 20 threads...



Total time in ppxf: 173.66 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.84 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.4094...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 55.40 s
Iteration 1: optimal regul = 10000.00; Δm = 2.79519e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 46.040
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 58.34 s
Iteration 2: optimal regul = 20000.00; Δm = 3.00029e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 36.926
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 61.50 s
Iteration 3: optimal regul = 30000.00; Δm = 2.95539e+09; Δregul = 500.00 (Δregul_min

In [34]:
df = pd.DataFrame()

In [35]:
###########################################################################
# Compute mean quantities from the ppxf runs
###########################################################################
thisrow = {}  # Row to append to DataFrame
thisrow["AGN continuum"] = False
thisrow["alpha_nu"] = np.nan
thisrow["log L_NT"] = np.nan

# Compute the mean SFH and SFR from the lists of MC runs
sfh_MC_lw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "lw")
sfh_MC_mw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "mw")
sfr_avg_MC = compute_mean_sfr(pp_mc_list, isochrones)
sfh_regul_mw_1D = pp_regul.sfh_mw_1D
sfh_regul_lw_1D = pp_regul.sfh_lw_1D
sfr_avg_regul = pp_regul.sfr_mean

# Compute the mean mass- and light-weighted ages plus the total mass in a series of age ranges
for aa in range(len(age_thresh_vals) - 1):
    age_thresh_lower = age_thresh_vals[aa]
    age_thresh_upper = age_thresh_vals[aa + 1]
    
    if age_thresh_lower is None:
        age_thresh_lower = ages[0]
    if age_thresh_upper is None:
        age_thresh_upper = ages[-1]
    age_str = f"{np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f}"
        
    # MC runs: compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_lw_mean, age_lw_std = compute_mean_age(pp_mc_list, isochrones, "lw", age_thresh_lower, age_thresh_upper)
    age_mw_mean, age_mw_std = compute_mean_age(pp_mc_list, isochrones, "mw", age_thresh_lower, age_thresh_upper)
    mass_mean, mass_std = compute_mean_mass(pp_mc_list, isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)
    
    # Regul run: compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_mw_regul = 10**compute_mw_age(sfh_regul_lw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    age_lw_regul = 10**compute_lw_age(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    mass_regul = compute_mass(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

    # Input: compute the mean mass- and light-weighted ages plus the total mass in this age range
    age_mw_input = 10**compute_mw_age(sfh_lw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    age_lw_input = 10**compute_lw_age(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
    mass_input = compute_mass(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

    # Put in DataFrame
    thisrow[f"MW age {age_str} (input)"] = age_mw_input
    thisrow[f"LW age {age_str} (input)"] = age_lw_input
    thisrow[f"Mass {age_str} (input)"] = mass_input
    thisrow[f"MW age {age_str} (MC) mean"] = age_mw_mean
    thisrow[f"LW age {age_str} (MC) mean"] = age_lw_mean
    thisrow[f"Mass {age_str} (MC) mean"] = mass_mean
    thisrow[f"MW age {age_str} (MC) std. dev."] = age_mw_std
    thisrow[f"LW age {age_str} (MC) std. dev."] = age_lw_std
    thisrow[f"Mass {age_str} (MC) std. dev."] = mass_std
    thisrow[f"MW age {age_str} (regularised)"] = age_mw_regul
    thisrow[f"LW age {age_str} (regularised)"] = age_lw_regul
    thisrow[f"Mass {age_str} (regularised)"] = mass_regul

df = df.append(thisrow, ignore_index=True)

In [39]:
###############################################################################
# The effect of the strength and exponent of the AGN continuum on the recovered SFH
###############################################################################
for aa, alpha_nu in enumerate(alpha_nu_vals):
    for ll, log_L_NT in enumerate(log_L_NT_vals):
        ###############################################################################
        # Run ppxf without an AGN continuum added as a "control"
        ###############################################################################
        # Create spectrum
        spec, spec_err, lambda_vals_A = create_mock_spectrum(
            sfh_mass_weighted=sfh_mw_input,
            agn_continuum=True, L_NT_erg_s=10**log_L_NT, alpha_nu=alpha_nu,
            isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
            plotit=False)

        ###########################################################################
        # Run ppxf WITHOUT regularisation, using a MC approach
        ###########################################################################
        # Input arguments
        seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
        args_list = [[s, spec, spec_err, lambda_vals_A] for s in seeds]

        # Run in parallel
        print(f"Running ppxf on {nthreads} threads...")
        t = time()
        with multiprocessing.Pool(nthreads) as pool:
            pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
        print(f"Total time in ppxf: {time() - t:.2f} s")

        ###########################################################################
        # Run ppxf with regularisation
        ###########################################################################
        t = time()
        pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                      z=z, ngascomponents=1,
                      regularisation_method="auto",
                      isochrones=isochrones,
                      fit_gas=False, tie_balmer=True,
                      delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
                      plotit=False, savefigs=False, interactive_mode=False)
        print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

        ###########################################################################
        # Compute mean quantities from the ppxf runs
        ###########################################################################
        thisrow = {}  # Row to append to DataFrame
        thisrow["AGN continuum"] = True
        thisrow["alpha_nu"] = alpha_nu
        thisrow["log L_NT"] = log_L_NT

        # Compute the mean SFH and SFR from the lists of MC runs
        sfh_MC_lw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "lw")
        sfh_MC_mw_1D_mean = compute_mean_1D_sfh(pp_mc_list, isochrones, "mw")
        sfr_avg_MC = compute_mean_sfr(pp_mc_list, isochrones)
        sfh_regul_mw_1D = pp_regul.sfh_mw_1D
        sfh_regul_lw_1D = pp_regul.sfh_lw_1D
        sfr_avg_regul = pp_regul.sfr_mean

        # Compute the mean mass- and light-weighted ages plus the total mass in a series of age ranges
        for aa in range(len(age_thresh_vals) - 1):
            age_thresh_lower = age_thresh_vals[aa]
            age_thresh_upper = age_thresh_vals[aa + 1]

            if age_thresh_lower is None:
                age_thresh_lower = ages[0]
            if age_thresh_upper is None:
                age_thresh_upper = ages[-1]
            age_str = f"{np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f}"

            # MC runs: compute the mean mass- and light-weighted ages plus the total mass in this age range
            age_lw_mean, age_lw_std = compute_mean_age(pp_mc_list, isochrones, "lw", age_thresh_lower, age_thresh_upper)
            age_mw_mean, age_mw_std = compute_mean_age(pp_mc_list, isochrones, "mw", age_thresh_lower, age_thresh_upper)
            mass_mean, mass_std = compute_mean_mass(pp_mc_list, isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

            # Regul run: compute the mean mass- and light-weighted ages plus the total mass in this age range
            age_mw_regul = 10**compute_mw_age(sfh_regul_lw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            age_lw_regul = 10**compute_lw_age(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            mass_regul = compute_mass(sfh_regul_mw_1D, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

            # Input: compute the mean mass- and light-weighted ages plus the total mass in this age range
            age_mw_input = 10**compute_mw_age(sfh_lw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            age_lw_input = 10**compute_lw_age(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)[0]
            mass_input = compute_mass(sfh_mw_input, isochrones=isochrones, age_thresh_lower=age_thresh_lower, age_thresh_upper=age_thresh_upper)

            # Put in DataFrame
            thisrow[f"MW age {age_str} (input)"] = age_mw_input
            thisrow[f"LW age {age_str} (input)"] = age_lw_input
            thisrow[f"Mass {age_str} (input)"] = mass_input
            thisrow[f"MW age {age_str} (MC) mean"] = age_mw_mean
            thisrow[f"LW age {age_str} (MC) mean"] = age_lw_mean
            thisrow[f"Mass {age_str} (MC) mean"] = mass_mean
            thisrow[f"MW age {age_str} (MC) std. dev."] = age_mw_std
            thisrow[f"LW age {age_str} (MC) std. dev."] = age_lw_std
            thisrow[f"Mass {age_str} (MC) std. dev."] = mass_std
            thisrow[f"MW age {age_str} (regularised)"] = age_mw_regul
            thisrow[f"LW age {age_str} (regularised)"] = age_lw_regul
            thisrow[f"Mass {age_str} (regularised)"] = mass_regul

        df = df.append(thisrow, ignore_index=True)

Running ppxf on 20 threads...



Total time in ppxf: 183.85 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.63 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3931...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 58.96 s
Iteration 1: optimal regul = 10000.00; Δm = 2.63113e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 43.149
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 61.39 s
Iteration 2: optimal regul = 20000.00; Δm = 9.45055e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 23.045
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 62.61 s
Iteration 3: optimal regul = 30000.00; Δm = 5.27867e+09; Δregul = 500.00 (Δregul_min


Total time in ppxf: 175.80 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 3.64 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.4126...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 57.46 s
Iteration 1: optimal regul = 8500.00; Δm = 2.94525e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 1.083
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 63.25 s
Iteration 2: optimal regul = 8400.00; Δm = 9.07374e+07; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.357
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.35653909864910815; using 8400.00 to produce the best fit
Total time in run_ppxf: 125.76 seconds
Running ppxf on 20 threads...



Total time in ppxf: 265.35 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 5.93 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.5691...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 86.12 s
Iteration 1: optimal regul = 0.00; Δm = 3.05882; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 85.89 s
Iteration 2: optimal regul = 100.00; Δm = 3.63303e+11; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 28.196
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 70.70 s
Iteration 3: optimal regul = 40.00; Δm = 7.85507e+10; Δregul = 20.00 (Δregul_min = 1.00); Δχ (


Total time in ppxf: 193.08 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.83 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3782...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 60.86 s
Iteration 1: optimal regul = 10000.00; Δm = 2.5507e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 51.857
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 65.70 s
Iteration 2: optimal regul = 20000.00; Δm = 5.84765e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 44.825
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 67.86 s
Iteration 3: optimal regul = 30000.00; Δm = 3.52356e+09; Δregul = 500.00 (Δregul_min 


Total time in ppxf: 198.64 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.32 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3922...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 61.53 s
Iteration 1: optimal regul = 10000.00; Δm = 3.2943e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 13.116
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 67.18 s
Iteration 2: optimal regul = 17000.00; Δm = 3.92924e+09; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.043
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.04252173071087384; using 17000.00 to produce the best fit
Total time in run_ppxf: 134.33 seconds
Running ppxf on 20 threads...



Total time in ppxf: 222.26 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 5.12 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.5864...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 79.00 s
Iteration 1: optimal regul = 500.00; Δm = 4.48507e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 53.032
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 82.78 s
Iteration 2: optimal regul = 100.00; Δm = 6.86619e+10; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 14.372
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 71.38 s
Iteration 3: optimal regul = 60.00; Δm = 3.24681e+10; Δregul = 20.00 (Δregul_min = 1.00)

In [74]:
###############################################################################
# Save DataFrame to file 
###############################################################################
# Add metadata 
df["SNR"] = SNR
df["niters"] = niters
df["nthreads"] = nthreads
df["z"] = z
df["Emission lines"] = False
df["isochrones"] = isochrones
df["sigma_star_kms"] = sigma_star_kms

# Save
df.to_hdf(os.path.join(data_path, f"ga{gal}_agncont.hd5"), key="agn")


### Load the DataFrames storing the output of each AGN continuum run and plot the derived age estimates
---

In [17]:
########################################################
# Plot the recovered mass-weighted mean age vs. alpha_nu
########################################################
pp = PdfPages("/priv/meggs3/u5708159/ppxftests/figs/agn_continuum/agn_cont.pdf")
for gal in range(1, 11):
    df = pd.read_hdf(os.path.join(data_path, f"ga{gal}_agncont.hd5"), key="agn")

    age_thresh_pairs = [
        (ages[0], 1e7),
        (ages[0], 1e8),
        (ages[0], 1e9),
        (1e9, ages[-1]),
        (ages[0], ages[-1]),
    ]

    alpha_nu_vals = [a for a in alpha_nu_vals if ~np.isnan(a)]
    cmap_alpha_nu = lambda aa: matplotlib.cm.get_cmap("Greens", len(alpha_nu_vals) + 1)(aa + 1)

    for age_thresh_pair in [((ages[0], 1e9))]:
        fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))
        fig.subplots_adjust(right=0.6)
        fig.subplots_adjust(hspace=0)

        age_thresh_lower, age_thresh_upper = age_thresh_pair
        age_str = f"{np.log10(age_thresh_lower):.2f} < log t < {np.log10(age_thresh_upper):.2f}"

        #************************************
        # Plot ages
        for weighttype, ax in zip(["LW", "MW"], axs):

            # Actual value
            ax.axhline(df[f"{weighttype} age {age_str} (input)"].unique()[0], color="gray", label="True value")

            # MC run w/o AGN continuum
            cond = df["AGN continuum"] == False
            ax.axhspan(ymin=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values[0] - df.loc[cond, f"{weighttype} age {age_str} (MC) std. dev."].values[0],
                       ymax=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values[0] + df.loc[cond, f"{weighttype} age {age_str} (MC) std. dev."].values[0],
                       color="pink", alpha=0.5)
            ax.axhline(df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values[0], color="pink", label="no AGN continuum")

            # Regul run w/o AGN continuum
            ax.axhline(df.loc[cond, f"{weighttype} age {age_str} (regularised)"].values[0], color="maroon", label="no AGN continuum (regularised)")

            for aa, alpha_nu in enumerate(alpha_nu_vals):
                cond = df["alpha_nu"] == alpha_nu
                # MC
                ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                            y=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values, 
                            yerr=df.loc[cond, f"{weighttype} age {age_str} (MC) std. dev."].values,
                            linestyle="none", marker="D", color=cmap_alpha_nu(aa),
                            label=r"$\alpha_\nu = %.1f$" % alpha_nu, zorder=999)
                # Regularised
                ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                            y=df.loc[cond, f"{weighttype} age {age_str} (MC) mean"].values, 
                            linestyle="none", marker="o", markerfacecolor="w", color=cmap_alpha_nu(aa),
                            label=r"$\alpha_\nu = %.1f$ (regularised)" % alpha_nu, zorder=999)

            # Decorations
            # ax.set_ylim([1e6, 1e10])
            ax.set_xlabel(r"$\log_{10} L_{\rm NT}$")
            ax.set_ylabel(f"{weighttype}-weighted mean age")
            ax.set_yscale("log")
            ax.grid()

        #************************************
        # Plot mass in each range
        ax = axs[2]

        # Actual value
        M_tot = df["Mass 6.60 < log t < 10.25 (input)"].unique()[0]
        ax.axhline(df[f"Mass {age_str} (input)"].unique()[0] / M_tot, color="gray", label="True value")

        # MC run w/o AGN continuum
        cond = df["AGN continuum"] == False
        ax.axhspan(ymin=(df.loc[cond, f"Mass {age_str} (MC) mean"].values[0] - df.loc[cond, f"Mass {age_str} (MC) std. dev."].values[0]) / M_tot,
                   ymax=(df.loc[cond, f"Mass {age_str} (MC) mean"].values[0] + df.loc[cond, f"Mass {age_str} (MC) std. dev."].values[0]) / M_tot,
                   color="pink", alpha=0.5)
        ax.axhline(df.loc[cond, f"Mass {age_str} (MC) mean"].values[0] / M_tot, color="pink", label="no AGN continuum (MC)")

        # Regul run w/o AGN continuum
        ax.axhline(df.loc[cond, f"Mass {age_str} (regularised)"].values[0] / M_tot, color="maroon", label="no AGN continuum (regularised)")

        # Results from MC runs
        for aa, alpha_nu in enumerate(alpha_nu_vals):
            cond = df["alpha_nu"] == alpha_nu
            # MC
            ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                        y=df.loc[cond, f"Mass {age_str} (MC) mean"].values / M_tot, 
                        yerr=df.loc[cond, f"Mass {age_str} (MC) std. dev."].values / M_tot,
                        linestyle="none", marker="D", color=cmap_alpha_nu(aa),
                        label=r"$\alpha_\nu = %.1f$" % alpha_nu, zorder=999)
            # Regularised
            ax.errorbar(x=df.loc[cond, "log L_NT"].values, 
                        y=df.loc[cond, f"Mass {age_str} (regularised)"].values / M_tot, 
                        linestyle="none", marker="o", markerfacecolor="w", color=cmap_alpha_nu(aa),
                        label=r"$\alpha_\nu = %.1f$ (regularised)" % alpha_nu, zorder=999)

        # Decorations
        # ax.set_ylim([1e8, 0.5e11])
        ax.set_xlabel(r"$\log_{10} L_{\rm NT}$")
        ax.set_ylabel(r"Mass fraction ($M/M_{\rm tot}$)")
        ax.set_yscale("log")
        ax.grid()

        axs[0].set_title(f"Galaxy ID {gal:004} ({age_str})" + r" $\log_{10}M_{\rm tot} = %.2f$" % np.log10(M_tot))    
        axs[1].legend(bbox_to_anchor=[1.05, 0.5], loc="center left", fontsize="small")
        
    pp.savefig(fig)
    plt.close("all")
    
pp.close()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …